In [ ]:
import pandas as pd
import json
import os
import snowflake.snowpark.functions as F
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session

In [ ]:
# Assuming you have environment variables set for the credentials
connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "role": os.getenv("SNOWFLAKE_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
}

session = Session.builder.configs(connection_parameters).create()


In [ ]:
#session = Session.builder.configs(SnowflakeLoginOptions()).getOrCreate()

In [ ]:
session.sql("CREATE TRANSIENT  DATABASE IF NOT EXISTS MLOPS;").collect()
session.sql("CREATE  TRANSIENT  SCHEMA IF NOT EXISTS MLOPS.advertising;").collect()
session.sql("USE DATABASE MLOPS").collect()
session.sql("USE SCHEMA MLOPS.advertising").collect()

In [ ]:
session.sql("CREATE STAGE if not exists ML_DATA").collect()

In [ ]:
ad = pd.read_csv(
    'https://raw.githubusercontent.com/HuseynA28/Snowflake-MLOPS/main/dataset/advertising.csv'
)
ad.columns = [c.upper() for c in ad.columns]
ad.to_csv("advertising.csv", index=False)

In [ ]:
session.file.put("advertising.csv", "@ml_data", overwrite=True)

Use the PUT command to upload a local CSV file to a Snowflake stage.

In [ ]:
# Create a Snowpark DataFrame that is configured to load data from the CSV file
avertising_df = (
    session.read.option("infer_schema", True)
    .option("PARSE_HEADER", True)
    .csv("@ml_data/advertising.csv")
)
avertising_df.show()

In [ ]:
def fix_values(column):
    return F.upper(F.regexp_replace(F.col(column), "[^a-zA-Z0-9]+", "_"))


for col in ["TV", "RADIO", "NEWSPAPER", "SALES"]:
    avertising_df = avertising_df.with_column(col, fix_values(col))

avertising_df.show()

In [ ]:
avertising_df.write.mode("overwrite").save_as_table("advertising")